<a href="https://colab.research.google.com/github/jachiaram/GuitarML/blob/main/GuitarML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Guitar Machine Learning Note Classifier Model V1**

## **Install Dependencies**

In [ ]:
!pip install tensorflow==2.8.0 tensorflow-gpu==2.8.0 tensorflow-io matplotlib

## **Import dependencies**

In [ ]:
import os
from matplotlib import pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/jachiaram/GuitarML/main/Guitar_features.csv')
print(df.columns)

Index(['note_names', 'chroma_stft', 'spectral_centroid', 'spectral_bandwidth',
       'spectral_rolloff'],
      dtype='object')


In [ ]:
y_column = 'note_names'
x_columns = ['chroma_stft', 'spectral_centroid', 'spectral_bandwidth', 'spectral_rolloff']

# Extract X and y
X = df[x_columns]
y = df[y_column]

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.head()

396     17. G-sharp 1
927     15. F-sharp 1
75      34. C-sharp 2
1154           16. G1
1654    29. G-sharp 2
Name: note_names, dtype: object

In [38]:
y_train.head

<bound method NDFrame.head of 396     17. G-sharp 1
927     15. F-sharp 1
75      34. C-sharp 2
1154           16. G1
1654    29. G-sharp 2
            ...      
2118           30. A2
1824             1. E
1186           25. E2
2211       3. F-sharp
1286       7. A-sharp
Name: note_names, Length: 1816, dtype: object>

In [39]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train_numeric = label_encoder.fit_transform(y_train)

y_train_one_hot = to_categorical(y_train_numeric, num_classes=37)

## **Creating, Compiling, and Training the Model:**
In this section the model is created, compiled, and then trained.


In [26]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score

In [27]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_dim=len(X_train.columns)))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=37, activation='softmax'))

In [28]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(X_train, y_train_one_hot, epochs=50, batch_size=32)

In [ ]:
from tensorflow.keras.utils import to_categorical
# Convert categorical test labels to one-hot encoding
y_test_one_hot = to_categorical(label_encoder.transform(y_test), num_classes=37)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test_one_hot)

print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')